In [1]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import animation
from celluloid import Camera
import seaborn as sns
sns.set_theme()
from IPython.display import HTML

In [2]:
def Lorenz96(x0, F):
    x = np.zeros(x0.shape[0]+3)
    # 周期的なxの表現をつくる
    x[2:-1] = x0 # index 2がもとのxのindex 0, index -2がもとのxのN
    x[:2] = x0[-2:] # index 0,1 がもとのxのindex N-1, N
    x[-1] = x0[0] # index -1 がもとのxのindex 0
    
    dxdt = (x[3:] - x[:-3]) * x[1:-2] - x[2:-1] + F
    return dxdt

def L96(x, F):
    """
    Lorenz 96 model with constant forcing.
    Cited by "https://en.wikipedia.org/wiki/Lorenz_96_model"
    
    Parameters
    ----------
    x : 
        variables
    F :
       Forcing constant 
    N : int
        number of sites
    """
    N = len(x)
    # Setting up vector
    d = np.zeros(N)
    # Loops over indices (with operations and Python underflow indexing handling edge cases)
    for i in range(N):
        d[i] = (x[(i + 1) % N] - x[i - 2]) * x[i - 1] - x[i] + F
    return d

In [3]:
def RK4_yield(initial, time, model, F) -> np.ndarray:
    """
    各時刻のRK4の結果をリターンするgenerator function
    
    Parameters
    ----------    
    F :
        Forcing constant, to be used in Lorenz96
    kwargs : 
        modelに渡す x 以外の変数を指定
        
    Return
    ----------
    generator
    """
    dt = time[1] - time[0]
    states = [initial]
    x = initial
    for i, t in enumerate(time[:-1]):
        if i == 0:
            yield x
        k1 = model(x, F)
        x1 = x + k1 * dt/2
        k2 = model(x1, F)
        x2 = x + k2 * dt/2
        k3 = model(x2, F)
        x3 = x + k3 * dt
        k4 = model(x3, F)
        x = x + (k1 + 2*k2 + 2*k3 + k4) * dt / 6
        yield x

# アニメーション

## F=8での挙動

In [4]:
# init
time = np.arange(0.0, 0.61, 0.05)
N = 40
F = 8.0
x1 = np.full(N, F)
x1[19] += F * 0.001

In [5]:
"""
matplotlib.animation.ArtistAnimationのラッパーcelluloidでアニメーションを作る

celluloid : https://github.com/jwkvam/celluloid
matplotlib.animation.ArtistAnimation : https://matplotlib.org/stable/api/_as_gen/matplotlib.animation.ArtistAnimation.html#matplotlib.animation.ArtistAnimation
"""
%matplotlib widget
fig = plt.figure()
camera = Camera(fig)
cnt = 0
for x in RK4_yield(x1, time, L96, F):
    t = plt.plot(x, color="green")
    plt.legend(t, [f'{cnt} unit time'], loc='upper left')
    cnt += 1
    camera.snap()
animation = camera.animate(interval=300, blit=True)

ModuleNotFoundError: No module named 'ipympl'

In [58]:
# save animation as gif
# animation.save('celluloid_subplots.gif', writer='pillow')